In [58]:
import re
import pandas as pd
import requests
import openpyxl
from time import sleep
from random import uniform
from tqdm import tqdm
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import Comment

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [3]:
def get_page(url):
    session = requests.Session()
    retries = Retry(total=3, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))
    
    try:
        response = session.get(url, headers=headers, verify=False, timeout=35)  # or verify=certifi.where()
        response.raise_for_status()
        return BeautifulSoup(response.text, 'html.parser')
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

In [13]:
def scrape_criminals(url):
    i = 0
    links = []
    while i < 50:
        link = f"{url}?page={i+1}"
        try:
            response = requests.get(link, headers=headers, verify=False)
            if response.status_code == 200:
                i += 1
                links.append(link)
                print(link)
            else:
                print(f"Non-200 status code {response.status_code} for {link}")
                break
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {link}: {e}")
            break
    soup_links = []
    for link in links:
        soup_links.append(get_page(link))
        sleep(uniform(1, 2))
    criminals = []
    for i in range(len(soup_links)):
        soup_result = soup_links[i].find("ul", class_="results")
        criminals.append(soup_result.find_all("li"))
    return criminals, links, soup_links

In [68]:
def scrape_criminal_data(criminals, links, url):
    pattern = r"^(?:[^/]*\/){2}(.*)"
    pattern_base_url = r'^(https?://[^/]+)'
    df = pd.DataFrame(columns=['Title', 'Text', 'Region'])
    match = re.match(pattern_base_url, url)
    if match:
        base_url = match.group(1)
    for i in tqdm(range(len(criminals))):
        for j in range(len(criminals[i])):
            title = ""
            full_text = ""
            region = ""
            a_tag = criminals[i][j].find("a")
            title = a_tag.text
            if a_tag:
                extracted_part = a_tag['href']
                full_path_to_criminal = f'{base_url}{extracted_part}'
                criminal_html = get_page(full_path_to_criminal)
                if criminal_html:
                    criminal_info_blocks = criminal_html.find("td", class_="h-col1 h-col1-inner3")
                    
                    region_info = criminal_info_blocks.find("div", class_="b-justice")
                    if region_info:
                        region = region_info.text
                    for comment in criminal_info_blocks.find_all(string=lambda text: isinstance(text, Comment)):
                        comment.extract()
                    text_element = criminal_info_blocks.find_all(
                        string=lambda text: (text.parent.name not in ['script', 'style', 'head', 'h3']) 
                        and len(text.strip()) > 20
                    )
                    full_text = "\n".join([p.strip() for p in text_element])
                    df.loc[len(df)] = title, full_text, region
            print(df)
            sleep(uniform(1, 2))
    return df

In [14]:
url = "https://sudact.ru/practice/po-moshennichestvu/"

In [15]:
criminals, links, soup_links = scrape_criminals(url)

D:\anacon\envs\tf-gpu\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sudact.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://sudact.ru/practice/po-moshennichestvu/?page=1


D:\anacon\envs\tf-gpu\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sudact.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://sudact.ru/practice/po-moshennichestvu/?page=2


D:\anacon\envs\tf-gpu\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sudact.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

D:\anacon\envs\tf-gpu\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sudact.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://sudact.ru/practice/po-moshennichestvu/?page=49


D:\anacon\envs\tf-gpu\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sudact.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://sudact.ru/practice/po-moshennichestvu/?page=50


D:\anacon\envs\tf-gpu\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sudact.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anacon\envs\tf-gpu\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sudact.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anacon\envs\tf-gpu\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sudact.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anacon\envs\tf-gpu\lib\site-packages\urllib3\connectionpool.py:1061: InsecureReque

In [69]:
df = scrape_criminal_data(criminals, links, url)

  0%|          | 0/50 [00:00<?, ?it/s]D:\anacon\envs\tf-gpu\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sudact.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


                                               Title  \
0  Апелляционное постановление № 22К-2834/2024 от...   

                                                Text  \
0  Апелляционное постановление № 22К-2834/2024 от...   

                                              Region  
0  Верховный Суд Республики Крым (Республика Крым...  


D:\anacon\envs\tf-gpu\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sudact.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


                                               Title  \
0  Апелляционное постановление № 22К-2834/2024 от...   
1  Апелляционное постановление № 22-2572/2024 22К...   

                                                Text  \
0  Апелляционное постановление № 22К-2834/2024 от...   
1  Апелляционное постановление № 22-2572/2024 22К...   

                                              Region  
0  Верховный Суд Республики Крым (Республика Крым...  
1  Иркутский областной суд (Иркутская область)   ...  


D:\anacon\envs\tf-gpu\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sudact.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

D:\anacon\envs\tf-gpu\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sudact.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


                                                 Title  \
0    Апелляционное постановление № 22К-2834/2024 от...   
1    Апелляционное постановление № 22-2572/2024 22К...   
2    Апелляционное постановление № 22-1027/2024 УК-...   
3    Апелляционное постановление № 22К-1005/2024 УК...   
4    Апелляционное постановление № 22-3615/2024 от ...   
..                                                 ...   
494  Апелляционное постановление № 22К-2051/2024 от...   
495  Апелляционное постановление № 22-1830/2024 от ...   
496  Апелляционное постановление № 22К-3636/2024 от...   
497  Апелляционное постановление № 22-969/2024 от 1...   
498  Апелляционное постановление № 22К-2048/2024 от...   

                                                  Text  \
0    Апелляционное постановление № 22К-2834/2024 от...   
1    Апелляционное постановление № 22-2572/2024 22К...   
2    Апелляционное постановление № 22-1027/2024 УК-...   
3    Апелляционное постановление № 22К-1005/2024 УК...   
4    Апелляци

D:\anacon\envs\tf-gpu\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sudact.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


                                                 Title  \
0    Апелляционное постановление № 22К-2834/2024 от...   
1    Апелляционное постановление № 22-2572/2024 22К...   
2    Апелляционное постановление № 22-1027/2024 УК-...   
3    Апелляционное постановление № 22К-1005/2024 УК...   
4    Апелляционное постановление № 22-3615/2024 от ...   
..                                                 ...   
495  Апелляционное постановление № 22-1830/2024 от ...   
496  Апелляционное постановление № 22К-3636/2024 от...   
497  Апелляционное постановление № 22-969/2024 от 1...   
498  Апелляционное постановление № 22К-2048/2024 от...   
499  Решение № 3А-172/2024 3А-172/2024~М-110/2024 М...   

                                                  Text  \
0    Апелляционное постановление № 22К-2834/2024 от...   
1    Апелляционное постановление № 22-2572/2024 22К...   
2    Апелляционное постановление № 22-1027/2024 УК-...   
3    Апелляционное постановление № 22К-1005/2024 УК...   
4    Апелляци

100%|██████████| 50/50 [20:31<00:00, 24.63s/it]


In [70]:
df.to_excel('sudact.xlsx')

In [71]:
df.to_csv('sudact.csv')